## SQP

In [ ]:
# QP

# Linesearch with penalty

# Active set strategy

# BFGS for approximating Hessian



In [1]:
# Import libraries
import torch as t
import numpy as np
from matplotlib import pyplot as plt

import torch.nn as nn
from torch.autograd import Variable
from torch.autograd.functional import jacobian

In [ ]:
# Part 1
"""
Decision variable: x0
State variables: x1
"""



m = 2 # No. of constraints
n = 2 # No. of variables
d = n - m # No. of decision variables


def f(x):
    # Define the objective function
    f = lambda x: x[0] ** 2 + (x[1] - 3) ** 2
    # Define the constraints
    g1 = lambda x: (x[1] ** 2) - (2 * x[0])
    g2 = lambda x: (x[1] - 1) ** 2 + (5 * x[0]) - 15
    
    return f(x), h1(x), h2(x)

# Reduced Gradient (Analytically computed)
# dfdd_analytical = lambda x: ((-5 * x[0] * x[2]) + (-21 * x[0] * x[2]) + (16 * x[1] * x[2])) / (10 * x[1] + 2 * x[2])

In [ ]:
# Compute Jacobian
def jac(x, n=n):
    J = t.zeros((n, n))
    for i in range(n):
       J[i] =  jacobian(f, (x))[i] # 'jacobian' function in Pytorch returns a tuple of tensors. Copying each tensor slice into a new tensor for the ease of indexing.
    return J

# Evaluate Constraints
def hFunc(x, m=m, n=n):
    H = t.zeros((m, 1))
    for i in range(m):
        H[i] =  f(x)[d + i]
    return H

# Part 3
# Calculate the Reduced gradient from the jacobian
dfdd = lambda J: J[0,0] - J[0,1:] @ t.pinverse(J[1:,1:]) @ J[1:,0]

In [ ]:
# Levenberg Marquardt solver
def LMSolve(x, m=m, n=n, epochs=60, Lambda=1.):      
    for i in range(epochs):
        H = hFunc(x)
        J =  jac(x)
        delta = t.pinverse(J[d:, d:].T @ J[d:, d:] + Lambda * t.eye(m)) @ J[d:, d:].T @ H # Change quantity in s_k
        with t.no_grad():
            x[d:] = x[d:] - delta.T # Update s_k
    return x


# Update point X
def updateX(x, alpha):
    xn = t.zeros(3)
    J = jac(x)
    xn[0] = x[0] - alpha * dfdd(J)
    xn[1:] = x[1:] + (alpha * t.pinverse(J[1:,1:]) @ J[1:,0].reshape(2,-1) *  dfdd(J)).T[0]
    return xn


# Armijo Line search
def lineSearch(x, t0=0.5, K=25):
    alpha = 1
    i = 0
    
    func = f(updateX(x, alpha))[0]
    phi = f(x)[0] - (t0 * alpha * dfdd(jac(x)) ** 2)
    
    while func > phi and i < K:
        alpha = 0.5 * alpha
        func = f(updateX(x, alpha))[0]
        phi = f(x)[0] - (t0 * alpha * dfdd(jac(x)) ** 2)

        i += 1
    return alpha

In [ ]:
# Part 4
# Initialization
def GRG(x):
    # Initialize variables
    e = t.norm(dfdd(jac(x)))
    tol = 1e-3 # Error threshold

    xSol = x.detach().numpy()
    fVal = [f(x)[0].item()]
    alphaSol = [1]
    eVal = [e]

    k = 0
    while e > tol:
        # Part 4.1
        # Inexact line search
        alpha = lineSearch(x)

        # Part 4.2 and 4.3
        J = jac(x)
        
        # Update the point      
        with t.no_grad():
            x = updateX(x, alpha)
        
        # Part 4.4
        # LM Solver
        x = LMSolve(x)

        # Part 4.5
        e = t.norm(dfdd(jac(x)))
        # Store important information in every iteration
        xSol = np.vstack((xSol, x.detach().numpy())) # Record x values in each iteration
        fVal.append(f(x)[0].item()) # Record f values in each iteration
        alphaSol.append(alpha) # Record alpha values in each iteration
        eVal.append(e)

        k += 1
        print (f"Iteration: {k:<5} Alpha: {alpha:<10} x: {str(x.detach().numpy()) :<40} f(x): {fVal[k]:<20} Error: {e:<20}")
    return xSol, fVal, alphaSol, eVal


In [ ]:
# Part 2
# Find the feasible point based on the initial gues of decision variables
x = t.tensor([0.1, 1., 1.], dtype=t.float64, requires_grad=True)
x = LMSolve(x)
print('Feasible starting point: ', x)
# print('Reduced gradient:', dfdd(jac(x)))
# print('Analytical calculation\n', dfdd_analytical(x))

# Find the optimal solution using GRG
print("\nGRG\n")
xSol, fVal, alphaSol, eVal = GRG(x)
print('\nThe optimal solution is x = ', xSol[-1, :])
# Convergence analysis

# Plot results
print("\nConvergence plot\n")
plt.semilogy(range(1, len(fVal)+1), eVal, "rs-")
# plt.xlim(1, 50)
plt.xticks(ticks=range(1, len(fVal)+1), labels=range(1, len(fVal)+1))
plt.xlabel("Iterations (k)")
plt.ylabel("Log|Error|")
plt.title("Convergence analysis")
plt.grid(axis='y', color='0.95')
plt.show()
